In [2]:
pip install pillow


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from tkinter import Tk, Text, filedialog, messagebox, Label, Button
from PIL import Image, ImageTk
import os

In [4]:
def encode_text(image_path, save_path, secret_message):
    binary_message = ''.join(format(ord(char), '08b') for char in secret_message)
    
    binary_message += '1111111111111110'

    img = Image.open(image_path)
    img = img.convert("RGBA")
    pixels = list(img.getdata())
    
    binary_index = 0
    for i in range(len(pixels)):
        if binary_index < len(binary_message):
            r, g, b, a = pixels[i]
            r = (r & ~1) | int(binary_message[binary_index])
            pixels[i] = (r, g, b, a)
            binary_index += 1
        else:
            break
    
    img.putdata(pixels)
    img.save(save_path)
    print(f"Encoding complete. Image saved as '{save_path}'.")


In [5]:
def decode_text(input_path):
    try:
        img = Image.open(input_path)
        img = img.convert("RGBA")
        pixels = list(img.getdata())
        
        binary_message = ""
        for pixel in pixels:
            r, g, b, a = pixel
            binary_message += str(r & 1)
        
        end_marker = "1111111111111110"

        if end_marker in binary_message:
            binary_message = binary_message.split(end_marker)[0]
        else:
            return None

        all_bytes = [binary_message[i:i+8] for i in range(0, len(binary_message), 8)]
        decoded_message = ""
        
        for byte in all_bytes:
            if len(byte) == 8:
                decoded_message += chr(int(byte, 2))
        
        if not decoded_message.strip() or "ÿ" in decoded_message:
            return None
        
        return decoded_message

    except Exception as e:
        raise RuntimeError(f"Decoding failed: {e}")


In [6]:
def browse_file():
    global selected_file
    selected_file = filedialog.askopenfilename(filetypes=[("Image Files", "*.png;*.bmp")])
    if selected_file:
        lbl_selected_file.config(text=f"Selected File: {os.path.basename(selected_file)}")

In [7]:
def display_images(original_path, encoded_path):
    original_image = Image.open(original_path).resize((200, 200))
    original_photo = ImageTk.PhotoImage(original_image)
    lbl_original_image.config(image=original_photo)
    lbl_original_image.image = original_photo
    lbl_original_text.config(text="Original Image")
    
    encoded_image = Image.open(encoded_path).resize((200, 200))
    encoded_photo = ImageTk.PhotoImage(encoded_image)
    lbl_encoded_image.config(image=encoded_photo)
    lbl_encoded_image.image = encoded_photo
    lbl_encoded_text.config(text="Stego-Image")

In [8]:
def encode_message():
    global selected_file
    if not selected_file:
        messagebox.showerror("Error", "Please select an image.")
        return
    
    secret_message = txt_message.get("1.0", "end-1c")
    if not secret_message:
        messagebox.showerror("Error", "Please enter a secret message.")
        return

    save_path = filedialog.asksaveasfilename(defaultextension=".png", filetypes=[("PNG Files", "*.png")])
    if not save_path:
        return  # User canceled the save dialog

    try:
        encode_text(selected_file, save_path, secret_message)
        display_images(selected_file, save_path)
        messagebox.showinfo("Success", "Message encoded successfully!")
    except RuntimeError as e:
        messagebox.showerror("Error", str(e))
    except Exception as e:
        messagebox.showerror("Unexpected Error", f"Something went wrong: {e}")


In [9]:
def decode_message():
    file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.png;*.bmp")])
    if not file_path:
        messagebox.showerror("Error", "No image selected for decoding.")
        return

    try:
        hidden_message = decode_text(file_path)

        if hidden_message:
            lbl_original_image.config(image="")
            lbl_original_image.image = None
            lbl_original_text.config(text="")
            lbl_encoded_text.config(text=f"Message Found: {hidden_message}")
        else:
            lbl_original_image.config(image="")
            lbl_original_image.image = None
            lbl_original_text.config(text="")
            lbl_encoded_text.config(text="No Hidden Message Found.")
    except RuntimeError as e:
        messagebox.showerror("Error", str(e))
    except Exception as e:
        messagebox.showerror("Unexpected Error", f"Something went wrong: {e}")


In [10]:
# GUI Setup
app = Tk()
app.title("Image Steganography")
app.geometry("500x400")

lbl_title = Label(app, text="Image Steganography", font=("Arial", 18))
lbl_title.pack(pady=10)

btn_browse = Button(app, text="Select Image", command=browse_file)
btn_browse.pack(pady=5)

lbl_selected_file = Label(app, text="Selected File: None", font=("Arial", 10))
lbl_selected_file.pack()

lbl_message = Label(app, text="Enter Secret Message:", font=("Arial", 12))
lbl_message.pack(pady=5)

txt_message = Text(app, height=5, width=40)
txt_message.pack(pady=5)

btn_encode = Button(app, text="Encode Message", command=encode_message, bg="lightblue")
btn_encode.pack(pady=5)

btn_decode = Button(app, text="Decode Message", command=decode_message, bg="lightgreen")
btn_decode.pack(pady=5)

lbl_original_text = Label(app, text="", font=("Arial", 12))
lbl_original_text.pack(pady=5)
lbl_original_image = Label(app)
lbl_original_image.pack()

lbl_encoded_text = Label(app, text="", font=("Arial", 12))
lbl_encoded_text.pack(pady=5)
lbl_encoded_image = Label(app)
lbl_encoded_image.pack()


In [11]:
app.mainloop()

Encoding complete. Image saved as 'C:/Users/Dell/Desktop/encoded.png'.
